In [2]:
import pickle
import pandas as pd
import glob2
import plotly.express as px
import plotly.graph_objects as go
from influxdb import InfluxDBClient
from plotly.subplots import make_subplots
import log_loader_time

Welcome to JupyROOT 6.16/00


In [28]:
# time={}
# with open ("/media/alb/Removibile/time_save/351","rb+") as savefile:
#     start_351,end_351=pickle.load(savefile)
# time["351"]=start_351

In [30]:
# dict_4_pd={
#     "run":[],
#     "subrun":[],
#     "time":[],
#     "Number_of_cluster_L1top_spX":[],
#     "Number_of_cluster_L1bot_spX":[],
#     "Number_of_cluster_L2top_spX":[],
#     "Number_of_cluster_L2bot_spX":[],
#     "Number_of_cluster_L1top_spV":[],
#     "Number_of_cluster_L1bot_spV":[],
#     "Number_of_cluster_L2top_spV":[],
#     "Number_of_cluster_L2bot_spV":[],
#     "Cluster_size_L1top_spX":[],
#     "Cluster_size_L1bot_spX":[],
#     "Cluster_size_L2top_spX":[],
#     "Cluster_size_L2bot_spX":[],
#     "Cluster_size_L1top_spV":[],
#     "Cluster_size_L1bot_spV":[],
#     "Cluster_size_L2top_spV":[],
#     "Cluster_size_L2bot_spV":[],
#     "Cluster_charge_L1top_spX":[],
#     "Cluster_charge_L1bot_spX":[],
#     "Cluster_charge_L2top_spX":[],
#     "Cluster_charge_L2bot_spX":[],
#     "Cluster_charge_L1top_spV":[],
#     "Cluster_charge_L1bot_spV":[],
#     "Cluster_charge_L2top_spV":[],
#     "Cluster_charge_L2bot_spV":[]
# }

# for filename, (run,subrun)in glob2.iglob("/home/alb/GRAAL_out/srv_lab_mount/716392*_*.txt", with_matches=True):
#     with open (filename,'r') as graal_file:
#         dict_4_pd["run"].append(int(run))
#         dict_4_pd["subrun"].append(int(subrun))
#         dict_4_pd["time"].append(time[run][subrun])
#         for line in graal_file.readlines():
#             for key in dict_4_pd:
#                 if "X" in key:
#                     if key.split("_sp")[0].replace("_"," ") in line:
#                         dict_4_pd["{}_spX".format(key.split("_sp")[0])].append(float(line.split("-->")[1].split("|")[0].strip("\n").strip(" fC ")))
#                         dict_4_pd["{}_spV".format(key.split("_sp")[0])].append(float(line.split("-->")[2].strip("\n").strip(" fC ")))

In [38]:
# cluster_pd=pd.DataFrame(dict_4_pd)
# cluster_pd=cluster_pd.sort_values(by=["run","subrun"])
# cluster_pd['time'] = pd.to_datetime(cluster_pd['time'],unit='s')
# cluster_pd=cluster_pd.set_index("time")
# cluster_pd["hour"]=cluster_pd.index.hour
# cluster_pd["day"]=cluster_pd.index.date

In [24]:
def extract_from_graal_and_save():
    dict_4_pd={
        "run":[],
        "subrun":[],
        "time":[],
        "Number_of_cluster_L1top_spX":[],
        "Number_of_cluster_L1bot_spX":[],
        "Number_of_cluster_L2top_spX":[],
        "Number_of_cluster_L2bot_spX":[],
        "Number_of_cluster_L1top_spV":[],
        "Number_of_cluster_L1bot_spV":[],
        "Number_of_cluster_L2top_spV":[],
        "Number_of_cluster_L2bot_spV":[],
        "Cluster_size_L1top_spX":[],
        "Cluster_size_L1bot_spX":[],
        "Cluster_size_L2top_spX":[],
        "Cluster_size_L2bot_spX":[],
        "Cluster_size_L1top_spV":[],
        "Cluster_size_L1bot_spV":[],
        "Cluster_size_L2top_spV":[],
        "Cluster_size_L2bot_spV":[],
        "Cluster_charge_L1top_spX":[],
        "Cluster_charge_L1bot_spX":[],
        "Cluster_charge_L2top_spX":[],
        "Cluster_charge_L2bot_spX":[],
        "Cluster_charge_L1top_spV":[],
        "Cluster_charge_L1bot_spV":[],
        "Cluster_charge_L2top_spV":[],
        "Cluster_charge_L2bot_spV":[]
    }
    time={}
    for filename, (run,)in glob2.iglob("/media/alb/Removibile/time_save/*", with_matches=True):
        with open (filename,"rb+") as savefile:
            start,end=pickle.load(savefile)
        time[int(run)]=start

        for filename, (subrun,)in glob2.iglob("/home/alb/GRAAL_out/srv_lab_mount/716392{}_*.txt".format(run), with_matches=True):
            with open (filename,'r') as graal_file:
                if subrun in time[int(run)].keys():
                    dict_4_pd["run"].append(int(run))
                    dict_4_pd["subrun"].append(int(subrun))
                    dict_4_pd["time"].append(time[int(run)][subrun])
                    for line in graal_file.readlines():
                        for key in dict_4_pd:
                            if "X" in key:
                                if key.split("_sp")[0].replace("_"," ") in line:
                                    dict_4_pd["{}_spX".format(key.split("_sp")[0])].append(float(line.split("-->")[1].split("|")[0].strip("\n").strip(" fC ")))
                                    dict_4_pd["{}_spV".format(key.split("_sp")[0])].append(float(line.split("-->")[2].strip("\n").strip(" fC ")))
                else:
                    print ("Missing, R{}, S{}".format(run,subrun))
        print ("RUN {} done".format(run))
        cluster_pd=pd.DataFrame(dict_4_pd)
        cluster_pd=cluster_pd.sort_values(by=["run","subrun"])
        # cluster_pd['time'] = pd.to_datetime(cluster_pd['time'],unit='s')
        # cluster_pd=cluster_pd.set_index("time")
        # cluster_pd["hour"]=cluster_pd.index.hour
        # cluster_pd["day"]=cluster_pd.index.date
        savefilename="/media/alb/Removibile/graal_extract/cluster_dataframe"
        with open (savefilename,"wb+") as savefile:
            pickle.dump(cluster_pd,savefile)  
    return cluster_pd

In [25]:
cluster_pd=extract_from_graal_and_save()

RUN 337 done
RUN 338 done
RUN 339 done
RUN 341 done
RUN 342 done
RUN 343 done
RUN 351 done
RUN 355 done
RUN 368 done
RUN 370 done
RUN 372 done
RUN 375 done
RUN 376 done
RUN 377 done
RUN 378 done
Missing, R387, S2
Missing, R387, S5
Missing, R387, S6
Missing, R387, S8
Missing, R387, S3
Missing, R387, S11
Missing, R387, S4
Missing, R387, S7
Missing, R387, S14
Missing, R387, S12
Missing, R387, S9
Missing, R387, S31
Missing, R387, S13
Missing, R387, S18
Missing, R387, S17
Missing, R387, S15
Missing, R387, S21
Missing, R387, S23
Missing, R387, S38
Missing, R387, S22
Missing, R387, S35
Missing, R387, S25
Missing, R387, S30
Missing, R387, S43
Missing, R387, S19
Missing, R387, S33
Missing, R387, S26
Missing, R387, S24
Missing, R387, S37
Missing, R387, S34
Missing, R387, S45
Missing, R387, S54
Missing, R387, S29
Missing, R387, S41
Missing, R387, S53
Missing, R387, S27
Missing, R387, S44
Missing, R387, S46
Missing, R387, S57
Missing, R387, S56
Missing, R387, S50
Missing, R387, S39
Missing, R387, 

In [27]:
cluster_pd['time'] = pd.to_datetime(cluster_pd['time'],unit='s')
cluster_pd=cluster_pd.set_index("time")
cluster_pd["hour"]=cluster_pd.index.hour
cluster_pd["day"]=cluster_pd.index.date

In [30]:
cluster_pd_1h=cluster_pd.resample("1h").mean()

In [17]:
# fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_size_L1top_spX,
#               mode="markers",name="Cluster_size_L1top_spX",
#               ))
# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_size_L1top_spV,
#               mode="markers",name="Cluster_size_L1top_spX",
#               ))
# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_size_L1bot_spX,
#           mode="markers",name="Cluster_size_L1bot_spX",
#           ))
# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_size_L1bot_spV,
#           mode="markers",name="Cluster_size_L1bot_spV",
#           ))
        
# fig.update_layout(title=dict(text='Hits', x=0.5),
#                    yaxis_title='Hits',
#                    xaxis_title='Subrun')
# fig.show()

In [18]:
# fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_size_L1top_spX,
#               mode="markers",name="Cluster_size_L2top_spX",
#               ))
# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_size_L1top_spV,
#               mode="markers",name="Cluster_size_L2top_spX",
#               ))
# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_size_L1bot_spX,
#           mode="markers",name="Cluster_size_L2bot_spX",
#           ))
# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_size_L1bot_spV,
#           mode="markers",name="Cluster_size_L2bot_spV",
#           ))
        
# fig.update_layout(title=dict(text='Hits', x=0.5),
#                    yaxis_title='Hits',
#                    xaxis_title='Subrun')
# fig.show()

In [39]:
# fig = make_subplots(specs=[[{"secondary_y": True}]])

# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_charge_L1top_spX,
#               mode="markers",name="Cluster_charge_L1top_spX",
#               ))

# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_charge_L1top_spV,
#               mode="markers",name="Cluster_charge_L1top_spV",
#               ))

# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_charge_L1bot_spX,
#           mode="markers",name="Cluster_charge_L1bot_spX",
#           ))

# fig.add_trace( 
#     go.Scatter(x=cluster_pd.subrun,y=cluster_pd.Cluster_charge_L1bot_spV,
#           mode="markers",name="Cluster_charge_L1bot_spV",
#           ))

        

# fig.update_layout(title=dict(text='Hits', x=0.5),
#                    yaxis_title='Hits',
#                    xaxis_title='Subrun')
# fig.show()


In [48]:
client = InfluxDBClient(host='localhost', port=8086)
client.get_list_database()
client.switch_database('GUFI_DB')
result=client.query('SELECT mean("Temperature") FROM "Environment" WHERE time > \'2020-01-5T00:00:00Z\' and time <\'2020-05-1T14:00:00Z\'  GROUP BY time(1h)',epoch='s')
points=result.get_points()
dict_4_pd={
    "time":[],
    "temperature":[]
}
for point in points:
    dict_4_pd["time"].append(point["time"]+28800)
    if point["mean"]==None:
            dict_4_pd["temperature"].append(0)
    else:
        dict_4_pd["temperature"].append(point["mean"])
temperature_pd_2=pd.DataFrame(dict_4_pd)
temperature_pd_2['time']=pd.to_datetime(temperature_pd_2['time'],unit='s')
temperature_pd_2=temperature_pd_2.set_index("time")

NameError: name 'InfluxDBClient' is not defined

In [ ]:
c

In [73]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
cluster_pd_1h["color_map"]= cluster_pd_1h["run"].astype(str)
fig.add_trace( 
    go.Scatter(x=cluster_pd_1h.index,
               y=cluster_pd_1h.Cluster_charge_L2top_spX,
               text=cluster_pd.run,
              mode="markers",
                hovertemplate=
                   'Run: %{text}'+
                    '<br><b>Cluster_charge_L2top_spX</b>: %{y}',
                name="Cluster_charge_L2top_spX",
                marker=dict(
                size=4,
                color=cluster_pd_1h.run,
                colorbar=dict(
                title="Colorbar")
        ),
              ))

# fig.add_trace( 
#     go.Scatter(x=cluster_pd.index[cluster_pd['Cluster_charge_L2top_spX'] > 1],y=cluster_pd.Cluster_charge_L2top_spV[cluster_pd['Cluster_charge_L2top_spX'] > 1],
#               mode="markers",name="Cluster_charge_L2top_spV",
#               ))

# fig.add_trace( 
#     go.Scatter(x=cluster_pd.index[cluster_pd['Cluster_charge_L2top_spX'] > 1],y=cluster_pd.Cluster_charge_L2bot_spX[cluster_pd['Cluster_charge_L2top_spX'] > 1],
#           mode="markers",name="Cluster_charge_L2bot_spX",
#           ))

# fig.add_trace( 
#     go.Scatter(x=cluster_pd.index[cluster_pd['Cluster_charge_L2top_spX'] > 1],y=cluster_pd.Cluster_charge_L2bot_spV[cluster_pd['Cluster_charge_L2top_spX'] > 1],
#           mode="markers",name="Cluster_charge_L2bot_spV",
#           ))

        

fig.update_layout(title=dict(text='Hits', x=0.5),
                   yaxis_title='Hits',
                   xaxis_title='Subrun')
fig.show()
